# **LOCAL OUTLIERS FACTORS (LOF)**

## **Penjelasan Local Outliers Factors**

LOF (Local Outlier Factor) adalah algoritma yang digunakan untuk mendeteksi outlier (anomali) dalam sekumpulan data. Outlier adalah titik data yang berbeda jauh dari data lainnya, misalnya transaksi mencurigakan dalam deteksi penipuan atau sensor yang memberikan pembacaan aneh.

Yang membedakan LOF dari metode lain adalah pendekatan berbasis kepadatan lokal. LOF membandingkan kepadatan suatu titik data dengan kepadatan tetangganya. Jika kepadatan suatu titik jauh lebih rendah dibandingkan dengan tetangganya, maka titik tersebut dianggap sebagai outlier.



## **Cara Kerja LOF (Langkah-Langkah)**  
1. Tentukan jumlah tetangga (k).
   - Kita memilih **k** tetangga terdekat untuk setiap titik (misalnya, \( k = 3 \)).  
   - Tetangga ini ditentukan menggunakan jarak rumus Euclidean, berikut adalah rumusnya: 
   $$
      d(A, B) = \sqrt{(x_B - x_A)^2 + (y_B - y_A)^2}
   $$



2. Hitung jarak jangkauan k-tetangga.
   - **Jarak jangkauan** dari titik \( A \) ke titik \( B \) didefinisikan sebagai:  

     $$ reach-dist_k(A, B) = \max(d(A, B), \text{k-distance}(B)) $$

     - $ d(A, B) $ = **jarak Euclidean** antara A dan B . 
     - $ {k-distance}(B) $ = jarak dari $B$ ke tetangga ke-$k$ terjauhnya.  
     - Jika $ A $ sudah lebih jauh dari tetangga ke- $k$  terjauh dari $ B $, gunakan jarak tersebut.  

3. Hitung kepadatan lokal (Local Reachability Density, LRD).
   - LRD mengukur seberapa "rapat" suatu titik dibandingkan dengan tetangganya.  
   - Rumusnya:  
     $$
     \text{LRD}_k(A) = \frac{k}{\sum_{B \in N_k(A)} \text{reach-dist}_k(A, B)}
     $$
     - $ N_k(A) $ = k tetangga terdekat dari $ A $.  
     - Semakin kecil LRD, semakin jarang titik tersebut berada dalam kelompoknya.  

4. Hitung Local Outlier Factor (LOF).
   - LOF membandingkan kepadatan titik $ A $ dengan kepadatan rata-rata tetangganya:  
     $$
     \text{LOF}_k(A) = \frac{\sum_{B \in N_k(A)} \frac{\text{LRD}_k(B)}{\text{LRD}_k(A)}}{k}
     $$
     - Jika **LOF ≈ 1**, maka titik memiliki kepadatan yang sama dengan tetangganya (bukan outlier).  
     - Jika **LOF > 1**, titik lebih jarang dibandingkan tetangganya (mungkin outlier).  
     - Jika **LOF ≫ 1** (misalnya lebih dari 1.5 atau 2), maka titik sangat berbeda dan kemungkinan besar merupakan **outlier**.  


## **Contoh Data**
| id  | x1 | x2 |
| --- | --- | --- |
| 1   | 2  | 2  |
| 2 | 3  | 5  |
| 3 | 1  | 1  |
| 4 | 20  | 40  |
| 5 | 5  | 3  |
| 6 | 2  | 3  |
| 7 | 4  | 1  |
| 8 | 1  | 5  |
| 9 | 5  | 4  |
| 10 | 3  | 4  |


## ***Proses Perhitungan LOF Manual tanpa Library scikit-learn***

Sebagai contoh dari perhitungan LOF, disini penulis hanya menggunakan data indeks ke-3 untuk proses perhitungan Euclidean, Reachibility Distance, dan juga Local Reachibility Dencity saja. Jika memungkinkan, hasil dari perhitungan yang lain akan ditampilkan dengan menggunakan excel. Urutan dari proses perhitungan LOF juga dimulai dengan menghitung rumus Euclidean, dilanjutkan dengan rumus RD, selanjutnya adalah rumus LRD dan yang terakhir adalah menentukan nilai LOF dari setiap data.


### *Menghitung Rumus Euclidean*

Proses pertama adalah menghitung jarak dari data indeks ke-3 ke semua data, dan untuk rumusnya kita menggunakan rumus Euclidean yang sudah penulis berikan di atas. berikut adalah perhitungannya

In [2]:
import numpy as np
from scipy.spatial.distance import euclidean


def compute_distance_between_points(data, point_indices):
    distances = {}
    for (i, j) in point_indices:
        if i != j:
            distances[(i, j)] = euclidean(data[i], data[j])
    return distances


points = np.array([
    [2, 2],
    [3, 5],
    [1, 1],
    [20, 40],
    [5, 3],
    [2,3],
    [4,1],
    [1,5],
    [5,4],
    [3,4]
])

selected_pairs = [(3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9)]
distances = compute_distance_between_points(points, selected_pairs)

sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
print("Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):")
for (i, j), dist in sorted_distances:
    print(f"Jarak dari indeks {i} ke indeks {j}: {dist:.2f}")


Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):
Jarak dari indeks 3 ke indeks 2: 43.38
Jarak dari indeks 3 ke indeks 6: 42.15
Jarak dari indeks 3 ke indeks 0: 42.05
Jarak dari indeks 3 ke indeks 5: 41.15
Jarak dari indeks 3 ke indeks 4: 39.92
Jarak dari indeks 3 ke indeks 7: 39.82
Jarak dari indeks 3 ke indeks 9: 39.81
Jarak dari indeks 3 ke indeks 8: 39.00
Jarak dari indeks 3 ke indeks 1: 38.91


In [3]:
points = np.array([
    [2, 2],
    [3, 5],
    [1, 1],
    [20, 40],
    [5, 3],
    [2,3],
    [4,1],
    [1,5],
    [5,4],
    [3,4]
])

selected_pairs = [(9, 0), (9, 1), (9, 2), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 3)]
distances = compute_distance_between_points(points, selected_pairs)

sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
print("Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):")
for (i, j), dist in sorted_distances:
    print(f"Jarak dari indeks {i} ke indeks {j}: {dist:.2f}")

Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):
Jarak dari indeks 9 ke indeks 3: 39.81
Jarak dari indeks 9 ke indeks 2: 3.61
Jarak dari indeks 9 ke indeks 6: 3.16
Jarak dari indeks 9 ke indeks 0: 2.24
Jarak dari indeks 9 ke indeks 4: 2.24
Jarak dari indeks 9 ke indeks 7: 2.24
Jarak dari indeks 9 ke indeks 8: 2.00
Jarak dari indeks 9 ke indeks 5: 1.41
Jarak dari indeks 9 ke indeks 1: 1.00


In [ ]:
points = np.array([
    [2, 2],
    [3, 5],
    [1, 1],
    [20, 40],
    [5, 3],
    [2,3],
    [4,1],
    [1,5],
    [5,4],
    [3,4]
])

selected_pairs = [(8, 0), (8, 1), (8, 2), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 3)]
distances = compute_distance_between_points(points, selected_pairs)


sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
print("Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):")
for (i, j), dist in sorted_distances:
    print(f"Jarak dari indeks {i} ke indeks {j}: {dist:.2f}")


Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):
Jarak dari indeks 8 ke indeks 3: 39.00
Jarak dari indeks 8 ke indeks 2: 5.00
Jarak dari indeks 8 ke indeks 7: 4.12
Jarak dari indeks 8 ke indeks 0: 3.61
Jarak dari indeks 8 ke indeks 5: 3.16
Jarak dari indeks 8 ke indeks 6: 3.16
Jarak dari indeks 8 ke indeks 1: 2.24
Jarak dari indeks 8 ke indeks 9: 2.00
Jarak dari indeks 8 ke indeks 4: 1.00


In [ ]:
points = np.array([
    [2, 2],
    [3, 5],
    [1, 1],
    [20, 40],
    [5, 3],
    [2,3],
    [4,1],
    [1,5],
    [5,4],
    [3,4]
])

selected_pairs = [(1, 0), (1, 9), (1, 2), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 3)]
distances = compute_distance_between_points(points, selected_pairs)

sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
print("Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):")
for (i, j), dist in sorted_distances:
    print(f"Jarak dari indeks {i} ke indeks {j}: {dist:.2f}")


Jarak Euclidean antara titik yang dipilih (diurutkan dari terbesar ke terkecil):
Jarak dari indeks 1 ke indeks 3: 38.91
Jarak dari indeks 1 ke indeks 2: 4.47
Jarak dari indeks 1 ke indeks 6: 4.12
Jarak dari indeks 1 ke indeks 0: 3.16
Jarak dari indeks 1 ke indeks 4: 2.83
Jarak dari indeks 1 ke indeks 5: 2.24
Jarak dari indeks 1 ke indeks 8: 2.24
Jarak dari indeks 1 ke indeks 7: 2.00
Jarak dari indeks 1 ke indeks 9: 1.00


Pada perhitungan di atas penulis juga mencari data jarak indeks-1 ke semua data, data jarak indeks-8 ke semua data, dan jarak indeks-9 ke semua data. Kenapa? Dikarenakan untuk melanjutkan ke proses perhitungan selanjutnya kita membutuhkan k-3 tetangga terdekat dari indeks-3. Bisa dilihat bahwa 3 tetangga terdekat dari indeks-3 adalah indeks-1, indeks-8 dan indeks-9. Oleh karena itu kita membutuhkan data k-3 dari masing masing data terdekat tersebut. berikut adalah data k-3 dari setiap indeksnya :

1. indeks-1 -> indeks-8 = 2.24 
2. indeks-8 -> indeks-1 = 2.24 
3. indeks-9 -> indeks-8 = 2.00

Data tersebutlah yang nanti akan kita butuhkan dan dibandingkan pada proses rumus RD(Reachibility Distance). Berikut ini adalah data lengkap mengenai semua jarak dari indeks-0 sampai dengan indeks-9, jadi pembaca dapat melihat sendiri bagaimana hasil perhitungannya.

![K-Distance](k-distance.png "a title")

### *Menghitung Rumus Reachibility Distance*

Setelah penulis mendapatkan data jarak pada indeks 3 ke semua data, dan juga mendapatkan data k-distance pada setiap data yang dekat dengan indeks 3. selanjutnya kita hanya perlu membandingkan nilai untuk mendapatkan RD-nya.


In [ ]:
k_distances_1 = 2.24
distance_3_1 = 38.91

k_distances_9 = 2.00
distance_3_9= 39.81

k_distances_8 = 2.24
distance_3_8= 39.00

rd3_1 = max(k_distances_1, distance_3_1)
rd3_8 = max(k_distances_8, distance_3_8)
rd3_9 = max(k_distances_9, distance_3_9)

print(f"Berikut ini adalah nilai dari RD(3,1) : {rd3_1}")
print(f"Berikut ini adalah nilai dari RD(3,8) : {rd3_8}")
print(f"Berikut ini adalah nilai dari RD(3,9) : {rd3_9}")


Berikut ini adalah nilai dari RD(3,1) : 38.91
Berikut ini adalah nilai dari RD(3,8) : 39.0
Berikut ini adalah nilai dari RD(3,9) : 39.81


Berikut ini adalah hasil perhitungan Excel dari rumus RD : 

![Reachibility Distance](RD.png "a title")

### *Menghitung Local Reachibility Distance*

Setelah kita mendapatkan RD untuk setiap jarak antar indeks 3, selanjutnya kita akan mendapatkan LRD 3 dengan rumus LRD

In [ ]:
# Menghitung LRD untuk titik (20,40)
rd_result = [rd3_1, rd3_8, rd3_9]
k = 3 
sum_rd = sum(rd_result)  
lrd_20_40 = k / sum_rd 

print(f"LRD dari index ke-3 = {lrd_20_40}")


LRD dari index ke-3 = 0.025484199796126403


Berikut ini adalah hasil perhitungan dari rumus LRD dengan perhitungan melalui excel : 


![Local Reachibility Distance](RD.png "a title")

setelah mendapatkan LRD kita akan menghitung nilai LOF pada setiap data yang penulis punya, jadi penulis membutuhkan semua data LRD untuk setiap data yang penulis punya. Oleh karena itu untuk mempersingkat prosesnya, penulis langsung menggunakan code yang akan memunculkan perhitungan hasil dari semua LRD tanpa menggunakan library scikit-learn. 

In [ ]:
data = np.array([
    [2, 2],
    [3, 5],
    [1, 1],
    [20, 40],
    [5, 3],
    [2,3],
    [4,1],
    [1,5],
    [5,4],
    [3,4]
])

def euclidean_distance(p1, p2):
    """Menghitung jarak Euclidean antara dua titik."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def compute_k_distance_neighbors(data, k):
    """Menghitung k tetangga terdekat untuk setiap titik."""
    num_points = len(data)
    neighbors_dict = {}
    
    for i in range(num_points):
        distances = [(j, euclidean_distance(data[i], data[j])) for j in range(num_points) if i != j]
        distances.sort(key=lambda x: x[1])
        neighbors_dict[i] = [j for j, d in distances[:k]]
    
    return neighbors_dict

def compute_rd(data, neighbors_dict, k):
    """Menghitung Reachability Distance (RD) untuk setiap titik."""
    rd_dict = {}
    
    k_distances = {i: max(euclidean_distance(data[i], data[j]) for j in neighbors_dict[i]) for i in range(len(data))}
    
    for i in range(len(data)):
        rd_dict[i] = []
        for neighbor in neighbors_dict[i]:
            rd = max(k_distances[neighbor], euclidean_distance(data[i], data[neighbor]))
            rd_dict[i].append(rd)
    
    return rd_dict

def compute_lrd(rd_dict, k):
    """Menghitung Local Reachability Density (LRD) untuk setiap titik."""
    lrd_dict = {}
    
    for i in rd_dict:
        sum_rd = sum(rd_dict[i])
        lrd_dict[i] = k / sum_rd if sum_rd != 0 else 0  # Hindari pembagian dengan nol
    
    return lrd_dict

# Langkah-langkah perhitungan
k = 3
neighbors_dict = compute_k_distance_neighbors(data, k)  
rd_dict = compute_rd(data, neighbors_dict, k)  
lrd_dict = compute_lrd(rd_dict, k) 

lrd_dict


{0: np.float64(0.37200096992612636),
 1: np.float64(0.4635254915624211),
 2: np.float64(0.40149162409079436),
 3: np.float64(0.0254837210798201),
 4: np.float64(0.41092720756334733),
 5: np.float64(0.4635254915624211),
 6: np.float64(0.41092720756334733),
 7: np.float64(0.4472135954999579),
 8: np.float64(0.4635254915624211),
 9: np.float64(0.4472135954999579)}

### *Menghitung Nilai Local Outliers Factor*

Setelah kita mendapatkan semua data yang dibutuhkan kita akan menghitung rumus LOF dari data LRD yang sudah tersedia. Dimana data yang kita gunakan adalah LRD dari indeks ke-1, LRD dari indeks ke-8, dan LRD dari indeks ke-9. berikut ini adalah perhitungannya

In [ ]:
# Data LRD yang sudah dihitung sebelumnya
lrd_3 = 0.0255

# LRD dari tetangga (1), (8), (9)
lrd_neighbors = [0.4635, 0.4635, 0.4472]

# Menghitung LOF untuk (3)
lof_20_40 = sum(lrd / lrd_20_40 for lrd in lrd_neighbors) / k

print(f"Nilai LOF dari indeks ke-3 adalah = {lof_20_40}")


Nilai LOF dari indeks ke-3 adalah = 17.974535999999997


Selanjutnya bisa kita bandingkan data nilai tersebut dengan data nilai LOF yang lain. untuk mencari nilainya disini saya akan menggunakan code python langsung agar tidak terlalu panjang. 

In [ ]:
# Menghitung LOF untuk semua titik dalam data
lof_dict = {}

for i in range(len(data)):
    lrd_i = lrd_dict[i]  # LRD titik i
    lrd_neighbors = [lrd_dict[j] for j in neighbors_dict[i]] 
    
    # Menghitung LOF
    lof_dict[i] = sum(lrd / lrd_i for lrd in lrd_neighbors) / k

lof_dict


{0: np.float64(1.1433163050353923),
 1: np.float64(0.9765393757777591),
 2: np.float64(1.034852364084868),
 3: np.float64(17.97571835918741),
 4: np.float64(1.072100906032108),
 5: np.float64(0.9224520048615017),
 6: np.float64(1.0110904040993238),
 7: np.float64(1.0243163389583858),
 8: np.float64(0.9504448828842479),
 9: np.float64(1.0364745084375788)}

## ***Proses Perhitungan LOF dengan Menggunakan Scikit-Learn***

Pada bab selanjutnya adalah penilaian dengan menggunakan library yang sudah ada di python yaitu scikit-learn. library dapat memudahkan kita mencari ouliers dengan hanya beberapa baris saja. Dapat dilihat dari code yang ada di bawah ini, yang harusnya codenya menghabiskan beberapa baris kolom kita dapat mempersingkatnya hanya dengan 1 kolom baris saja. 


In [ ]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

data = np.array([
    [2, 2], [3, 5], [1, 1], [20, 40], [5, 3],
    [2, 3], [4, 1], [1, 5], [5, 4], [3, 4]
])

lof_model = LocalOutlierFactor(n_neighbors=3, metric='euclidean')

lof_scores = -lof_model.fit_predict(data)
lof_values = -lof_model.negative_outlier_factor_

for i, point in enumerate(data):
    print(f"Indeks-{(i)} → LOF: {lof_values[i]:.4f}")


Indeks-0 → LOF: 1.0995
Indeks-1 → LOF: 0.9413
Indeks-2 → LOF: 0.9942
Indeks-3 → LOF: 17.9757
Indeks-4 → LOF: 1.0721
Indeks-5 → LOF: 0.9815
Indeks-6 → LOF: 0.9714
Indeks-7 → LOF: 0.9878
Indeks-8 → LOF: 0.9504
Indeks-9 → LOF: 1.0000
